In [1]:
# Summary: the code is FEniCS-based implementation of Stokes and contiuum equations 
#    with mechanical anisotropy described in Maulhaus et al. (2002). 
# Part of the code is adopted from the FEniCS Stokes-iterative demo, whose link is provided below:
#    https://fenicsproject.org/olddocs/dolfin/1.3.0/python/demo/documented/stokes-iterative/python/documentation.html

# Now version 1.1.9. 
# Created on 08/02/2021. Last modfied on 06/29/2021. 
# Author: Dunyu Liu (dliu@ig.utexas.edu) 

In [2]:
from dolfin import *
import numpy as np
from lib import *
from prepare_case import *

%matplotlib inline
#sys.setrecursionlimit(1500)

# Test for PETSc or Epetra
if not has_linear_algebra_backend("PETSc") and not has_linear_algebra_backend("Epetra"):
    info("DOLFIN has not been configured with Trilinos or PETSc. Exiting.")
    exit()

if not has_krylov_solver_preconditioner("amg"):
    info("Sorry, this code is only available when DOLFIN is compiled with AMG "
	 "preconditioner, Hypre or ML.");
    exit()

In [3]:
case = 26
case_info = {26:'2D rectangle with a horizontal anisotropic layer. Same as the analytic solution.',
              30:'3D box with a horziontal anisotropic layer.',
              31:'3D box with vertical SAF fault zone parallel to boundaries.',
              32:'3D Leech River Schist above the Cascadia subduction zone.'}
# case 26 for 2D rectangle with a horizontal anisotropic layer. Same as the analytic solution.
# case 30 for 3D box with a horziontal anisotropic layer.
# case 31 for 3D box with vertical SAF fault zone parallel to boundaries.
# case 32 for 3D Leech River Schist above the Cascadia subduction zone. 

date_stamp = '20220629' # Date stamp is used in the output folder name.

In [4]:
def stokes(delta, alpha, eta_strong, eta_weak, msh_path = None, msh_name = None, n = 10):
    
    # Use function prepare_case to generate key model specific data.
    mesh, boundaries, mf, bcs, f, bound_name_list, dim, W, norm1, norm2, norm3 = prepare_case(case, msh_path, msh_name, n, delta, alpha)
    #------- Case specific modifications end here --------
    
    anis_domain = bound_name_list['anis_domain']
    iso_domain = bound_name_list['iso_domain']
    
    ds = Measure("ds")(domain=mesh, subdomain_data = boundaries)
    
    if eta_strong>eta_weak:
        hete = 1
        iso = False
    elif eta_strong == eta_weak: 
        hete = 0
        iso = True

    # Define eta and eta_s.
    # eta is the strong viscosity in the anisotropic cases or the viscosity of isotropic case.
    # eta_s is the weak viscosity in the anisotropic cases.
    if iso == True:
        eta = K(mf, eta_weak, eta_strong, anis_domain, iso_domain, degree=0)
    else:
        eta = eta_strong
    eta_s = K(mf, eta_weak, eta_strong, anis_domain, iso_domain, degree=0) 
    
    # Define functions -----------------------------------
    def epsilon(v):
        return sym(nabla_grad(v))
    # Define stress
    def sigma(v,p,eta):
        return 2*eta*sym(nabla_grad(v))-(-p)*Identity(dim)
    
    # https://www.continuummechanics.org/principalstressesandstrains.html
    # calc_princ is derived from the above link.
    def calc_princ(sigma):
        thetap = atan(2*sigma[0,1]/(sigma[0,0] - sigma[1,1]))
        thetap = thetap/math.pi*180/2

        sigma1 = (sigma[0,0] + sigma[1,1])/2 + sqrt(pow((sigma[0,0]/2 - sigma[1,1]/2),2) + sigma[0,1]*sigma[0,1])
        sigma3 = (sigma[0,0] + sigma[1,1])/2 - sqrt(pow((sigma[0,0]/2 - sigma[1,1]/2),2) + sigma[0,1]*sigma[0,1])

        return thetap, sigma1, sigma3
    
    # Define anisotropic viscosity constitutive relation
    # Based on Maulhaus et al. (2002). 
    def sigma_anisotropic(v, eta, eta_s, dim):
        if dim == 2:
            n = np.zeros((1,dim))
            n[0][0] = norm1
            n[0][1] = norm3       
        elif dim == 3:
            n = np.zeros((1,dim))
            n[0][0] = norm1
            n[0][1] = norm2
            n[0][2] = norm3
        C_lambda = np.zeros((dim, dim, dim, dim))

        componentList = []
        for i in range(dim):
            componentList += [[],]
            for j in range(dim):
                componentList[i] += [[],]
                for k in range(dim):
                    componentList[i][j] += [[],]
                    for l in range(dim):
                        componentList[i][j][k] += [[],]    

        for i in range(dim):
            for j in range(dim):
                for k in range(dim):
                    for l in range(dim):
                        a = 0
                        b = 0
                        c = 0
                        d = 0
                        e = n[0][i]*n[0][j]*n[0][k]*n[0][l]
                        if l == j :
                            a = n[0][i]*n[0][k]
                        if i == l :
                            b = n[0][j]*n[0][k]
                        if k == j :
                            c = n[0][i]*n[0][l]
                        if i == k :
                            d = n[0][j]*n[0][l]
                        componentList[i][j][k][l] = 2*(eta_s - eta) * ((a + b + c + d)/2 - 2*e)
                        #C_lambda[i][j][k][l] = (a + b + c + d)/2 - 2*e

        C = as_tensor(componentList)
        vtmp = as_tensor(epsilon(v))
        i, j, k, l = ufl.indices(4)
        C1 = ufl.as_tensor(C[i,j,k,l]*vtmp[k,l],(i,j))
        C2 = 2*eta*sym(nabla_grad(v))
        C3 = C1 + C2
        return C3

    # Define variational problem
    (u, p) = TrialFunctions(W)
    (v, q) = TestFunctions(W)
    
    if iso:
        theta2 = -10000
        nametag = '_isotropic' # Appended in the result files.
        a = inner(grad(u), eta*grad(v))*dx + div(v)*p*dx + q*div(u)*dx
        L = inner(f, v)*dx
        # b is used to build the preconditioner matrix
        b = inner(grad(u), eta*grad(v))*dx + p*q*dx
    else:
        nametag = '_anisotropic'
        a1 = inner(sigma_anisotropic(u,eta,eta_s,dim), grad(v))*dx
        a2 = div(v)*p*dx + q*div(u)*dx
        a = a1 + a2
        L = inner(f, v)*dx
        # b is used to build the preconditioner matrix
        b = inner(grad(u), 2*eta*grad(v))*dx + p*q*dx    

    # Assemble the system
    A, bb = assemble_system(a, L, bcs)
    # Assemble the preconditioner system
    P, btmp = assemble_system(b, L, bcs)
    print('Assembling the system and preconditioner ...')

    # Create Krylov solver and AMG preconditioner
    solver = KrylovSolver("minres", "amg") # best.
    #solver = KrylovSolver("gmres", "amg") # second best.
    #solver = KrylovSolver("tfqmr", "amg") # least

    solver.parameters["relative_tolerance"] = 1.0e-9
    solver.parameters["absolute_tolerance"] = 1.0e-15
    solver.parameters["divergence_limit"] = 1.0e4
    solver.parameters["maximum_iterations"] = 7000
    solver.parameters["error_on_nonconvergence"] = True
    solver.parameters["nonzero_initial_guess"] = False
    solver.parameters["report"] = True
    solver.parameters["monitor_convergence"] = True
    # Associate operator (A) and preconditioner matrix (P)
    solver.set_operators(A, P)

    # Solve
    U = Function(W)
    
    print('Solving the system, be patient...')

    solver.solve(U.vector(), bb)
    
    # Get sub-functions
    u, p = U.split()

    # Calculate stress and strain-rate tensors
    Vsig = TensorFunctionSpace(mesh, "DG", degree=0) # Set DG0 for stress & strain rate
    sig = Function(Vsig, name="Stress")
    strain_rate = Function(Vsig, name="Strain")

    # Project stresses to Vsig space.
    if iso==True:
        sig.assign(project(sigma(u,p,eta), Vsig))
    else: 
        sig.assign(project(sigma_anisotropic(u,eta,eta_s,dim)-(-p)*Identity(dim), Vsig))

    strain_rate.assign(project(epsilon(u), Vsig))

    print('Writing out the results ...')
    # Save solution in xdmf format
    ufile_pvd = XDMFFile("../res/case"+ str(case) + "/" + str(date_stamp) + "/velocity_theta"+"{:.1f}".format(delta) + "_hetero_"+ str(hete) + ".xdmf")
    ufile_pvd.write(u)
    pfile_pvd = XDMFFile("../res/case"+ str(case) + "/" + str(date_stamp) + "/pressure_theta"+"{:.1f}".format(delta) + "_hetero_"+ str(hete) + ".xdmf")
    pfile_pvd.write(p)
    pfile_pvd = XDMFFile("../res/case"+ str(case) + "/" + str(date_stamp) + "/stress_theta"+"{:.1f}".format(delta) + "_hetero_"+ str(hete) + ".xdmf")
    pfile_pvd.write(sig)
    pfile_pvd = XDMFFile("../res/case" + str(case) + "/" + str(date_stamp) + "/strain_rate_theta"+"{:.1f}".format(delta) + "_hetero_"+ str(hete) + ".xdmf")
    pfile_pvd.write(strain_rate)
    
    print('Finished ...')

In [5]:
# theta is the angle of the layer normal to the horizontal axis in the square model.
# theta should be >90 and <180.
# delta = theta - 90.

print(case_info[case])
if case == 26:
    n= 40
    eta_strong, eta_weak = 1, 0.01
    msh_path = None
    msh_name = None
    alpha = -999
    for delta in np.arange(0,90+22.5,22.5):
        stokes(delta, alpha, eta_strong, eta_weak, msh_path, msh_name, n)
        
if case == 30: 
    # Figure 4b & 6.
    n = 10
    eta_strong, eta_weak = 1, 0.1
    msh_path = None
    msh_name = None
    delta = 10
    alpha = 45
    stokes(delta, alpha, eta_strong, eta_weak, msh_path, msh_name, n)
        
if case == 31:
    # Figure 4a & 5.
    n = 10
    eta_strong, eta_weak = 1, 0.1
    msh_path = None
    msh_name = None
    delta = 12.5
    stokes(delta, alpha, eta_strong, eta_weak, msh_path, msh_name, n)
        
if case == 32:
    # Figure 7 & 8.
    msh_path = "../msh/Cascadia_schist_mesh_case32/"
    msh_name = "cascadia_thrust"
    eta_strong, eta_weak = 1, 0.1
    delta = 30
    alpha = -999
    n = -999
    stokes(delta, alpha, eta_strong, eta_weak, msh_path, msh_name, n)

2D rectangle with a horizontal anisotropic layer. Same as the analytic solution.
Simulating Case 26: Same as 2D analytic solution. Horizontal anisotropic layer subjected to simple shear.
Assembling the system and preconditioner ...
Solving the system, be patient...
Solving linear system of size 145000 x 145000 (PETSc Krylov solver).
  0 KSP preconditioned resid norm 5.824541540267e+01 true resid norm 9.021579068483e+01 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP preconditioned resid norm 1.450281693862e+01 true resid norm 2.074033022724e+01 ||r(i)||/||b|| 2.298968957630e-01
  2 KSP preconditioned resid norm 2.737642092369e+00 true resid norm 4.363156875160e+00 ||r(i)||/||b|| 4.836356076956e-02
  3 KSP preconditioned resid norm 9.030934973388e-01 true resid norm 1.049901550244e+00 ||r(i)||/||b|| 1.163766943984e-02
  4 KSP preconditioned resid norm 4.684585322028e-01 true resid norm 5.441092143035e-01 ||r(i)||/||b|| 6.031197090589e-03
  5 KSP preconditioned resid norm 2.521808331404e-01 tr

106 KSP preconditioned resid norm 3.989863854906e-05 true resid norm 3.679713521292e-05 ||r(i)||/||b|| 4.078790967035e-07
107 KSP preconditioned resid norm 3.672197282182e-05 true resid norm 3.430327124668e-05 ||r(i)||/||b|| 3.802357767557e-07
108 KSP preconditioned resid norm 3.584238499561e-05 true resid norm 3.099894769981e-05 ||r(i)||/||b|| 3.436088900235e-07
109 KSP preconditioned resid norm 3.558238039996e-05 true resid norm 3.223506238033e-05 ||r(i)||/||b|| 3.573106452388e-07
110 KSP preconditioned resid norm 3.326647742629e-05 true resid norm 3.075798312922e-05 ||r(i)||/||b|| 3.409379100458e-07
111 KSP preconditioned resid norm 3.091294935474e-05 true resid norm 2.743637780998e-05 ||r(i)||/||b|| 3.041194629200e-07
112 KSP preconditioned resid norm 3.091294784085e-05 true resid norm 2.743271855454e-05 ||r(i)||/||b|| 3.040789017787e-07
113 KSP preconditioned resid norm 2.852318920224e-05 true resid norm 2.857136848642e-05 ||r(i)||/||b|| 3.167003056730e-07
114 KSP preconditioned r

196 KSP preconditioned resid norm 1.003318907643e-06 true resid norm 1.039378704065e-06 ||r(i)||/||b|| 1.152102859350e-08
197 KSP preconditioned resid norm 9.152270894465e-07 true resid norm 8.963338290686e-07 ||r(i)||/||b|| 9.935442811780e-09
198 KSP preconditioned resid norm 9.039324350733e-07 true resid norm 8.382946088176e-07 ||r(i)||/||b|| 9.292105101048e-09
199 KSP preconditioned resid norm 8.751001896405e-07 true resid norm 8.669242494916e-07 ||r(i)||/||b|| 9.609451326766e-09
200 KSP preconditioned resid norm 7.877031082267e-07 true resid norm 8.018770697334e-07 ||r(i)||/||b|| 8.888433650544e-09
201 KSP preconditioned resid norm 7.335893801313e-07 true resid norm 7.066702772616e-07 ||r(i)||/||b|| 7.833110721496e-09
202 KSP preconditioned resid norm 7.309108242275e-07 true resid norm 7.355987732946e-07 ||r(i)||/||b|| 8.153769619605e-09
203 KSP preconditioned resid norm 6.738869430768e-07 true resid norm 7.174401959779e-07 ||r(i)||/||b|| 7.952490251782e-09
204 KSP preconditioned r

 27 KSP preconditioned resid norm 1.872682627130e-03 true resid norm 1.717493064073e-03 ||r(i)||/||b|| 1.903761027904e-05
 28 KSP preconditioned resid norm 1.802137936809e-03 true resid norm 1.946559329149e-03 ||r(i)||/||b|| 2.157670308460e-05
 29 KSP preconditioned resid norm 1.417181956207e-03 true resid norm 1.527299824142e-03 ||r(i)||/||b|| 1.692940684273e-05
 30 KSP preconditioned resid norm 1.259089256853e-03 true resid norm 1.127683052551e-03 ||r(i)||/||b|| 1.249984114744e-05
 31 KSP preconditioned resid norm 1.252819507463e-03 true resid norm 1.188863644535e-03 ||r(i)||/||b|| 1.317799949998e-05
 32 KSP preconditioned resid norm 8.713588145860e-04 true resid norm 1.060725859209e-03 ||r(i)||/||b|| 1.175765186069e-05
 33 KSP preconditioned resid norm 7.298829907956e-04 true resid norm 7.373604752107e-04 ||r(i)||/||b|| 8.173297264407e-06
 34 KSP preconditioned resid norm 7.044847901966e-04 true resid norm 8.503390014814e-04 ||r(i)||/||b|| 9.425611581148e-06
 35 KSP preconditioned r

reconditioned resid norm 4.396055830599e-07 true resid norm 4.479416555942e-07 ||r(i)||/||b|| 4.965224515508e-09
143 KSP preconditioned resid norm 4.236635834897e-07 true resid norm 4.734582990608e-07 ||r(i)||/||b|| 5.248064617809e-09
144 KSP preconditioned resid norm 3.867800964852e-07 true resid norm 4.288476284866e-07 ||r(i)||/||b|| 4.753576122664e-09
145 KSP preconditioned resid norm 3.713701902852e-07 true resid norm 3.736650175442e-07 ||r(i)||/||b|| 4.141902594964e-09
146 KSP preconditioned resid norm 3.678635382756e-07 true resid norm 3.937114035877e-07 ||r(i)||/||b|| 4.364107442822e-09
147 KSP preconditioned resid norm 3.299146745997e-07 true resid norm 3.618623414068e-07 ||r(i)||/||b|| 4.011075429921e-09
148 KSP preconditioned resid norm 3.077849728923e-07 true resid norm 3.063151679397e-07 ||r(i)||/||b|| 3.395360896518e-09
149 KSP preconditioned resid norm 3.076453785098e-07 true resid norm 3.023047812066e-07 ||r(i)||/||b|| 3.350907628385e-09
150 KSP preconditioned resid norm

 69 KSP preconditioned resid norm 3.430898596748e-05 true resid norm 6.210532658800e-05 ||r(i)||/||b|| 6.884086047083e-07
 70 KSP preconditioned resid norm 3.410334266741e-05 true resid norm 6.069629351887e-05 ||r(i)||/||b|| 6.727901297336e-07
 71 KSP preconditioned resid norm 3.356154762220e-05 true resid norm 6.087157170144e-05 ||r(i)||/||b|| 6.747330067094e-07
 72 KSP preconditioned resid norm 3.233337573944e-05 true resid norm 5.804805295189e-05 ||r(i)||/||b|| 6.434356171047e-07
 73 KSP preconditioned resid norm 3.222875191259e-05 true resid norm 5.711151029420e-05 ||r(i)||/||b|| 6.330544781647e-07
 74 KSP preconditioned resid norm 3.147661172483e-05 true resid norm 5.697506031396e-05 ||r(i)||/||b|| 6.315419937182e-07
 75 KSP preconditioned resid norm 3.005453631421e-05 true resid norm 5.368284337703e-05 ||r(i)||/||b|| 5.950493031156e-07
 76 KSP preconditioned resid norm 3.004272204454e-05 true resid norm 5.345368110785e-05 ||r(i)||/||b|| 5.925091461492e-07
 77 KSP preconditioned r

|| 6.026256220064e-09
184 KSP preconditioned resid norm 2.807102818442e-07 true resid norm 5.476529613691e-07 ||r(i)||/||b|| 6.070477875457e-09
185 KSP preconditioned resid norm 2.649490364342e-07 true resid norm 5.241312365967e-07 ||r(i)||/||b|| 5.809750517265e-09
186 KSP preconditioned resid norm 2.489263175018e-07 true resid norm 4.848704874553e-07 ||r(i)||/||b|| 5.374563408187e-09
187 KSP preconditioned resid norm 2.487509490071e-07 true resid norm 4.820071282300e-07 ||r(i)||/||b|| 5.342824405474e-09
188 KSP preconditioned resid norm 2.347981860253e-07 true resid norm 4.670584800806e-07 ||r(i)||/||b|| 5.177125606672e-09
189 KSP preconditioned resid norm 2.165027842640e-07 true resid norm 4.298858027777e-07 ||r(i)||/||b|| 4.765083800901e-09
190 KSP preconditioned resid norm 2.128443014548e-07 true resid norm 4.146517573340e-07 ||r(i)||/||b|| 4.596221506084e-09
191 KSP preconditioned resid norm 2.079629495314e-07 true resid norm 4.150591346583e-07 ||r(i)||/||b|| 4.600737093890e-09
19

 79 KSP preconditioned resid norm 1.306648897952e-05 true resid norm 1.252309854107e-05 ||r(i)||/||b|| 1.388127116773e-07
 80 KSP preconditioned resid norm 1.285683307215e-05 true resid norm 1.325659272737e-05 ||r(i)||/||b|| 1.469431529308e-07
 81 KSP preconditioned resid norm 1.222181335968e-05 true resid norm 1.284874278677e-05 ||r(i)||/||b|| 1.424223263936e-07
 82 KSP preconditioned resid norm 1.205544227973e-05 true resid norm 1.163579635573e-05 ||r(i)||/||b|| 1.289773804276e-07
 83 KSP preconditioned resid norm 1.172871270725e-05 true resid norm 1.240767292973e-05 ||r(i)||/||b|| 1.375332725629e-07
 84 KSP preconditioned resid norm 1.070549229245e-05 true resid norm 1.136723623693e-05 ||r(i)||/||b|| 1.260005166572e-07
 85 KSP preconditioned resid norm 1.052814519033e-05 true resid norm 1.029793811492e-05 ||r(i)||/||b|| 1.141478452579e-07
 86 KSP preconditioned resid norm 1.009827313827e-05 true resid norm 1.131758562266e-05 ||r(i)||/||b|| 1.254501627348e-07
 87 KSP preconditioned r

Assembling the system and preconditioner ...
Solving the system, be patient...
Solving linear system of size 145000 x 145000 (PETSc Krylov solver).
  0 KSP preconditioned resid norm 5.824541540267e+01 true resid norm 9.021579068483e+01 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP preconditioned resid norm 1.450281693862e+01 true resid norm 2.074033022724e+01 ||r(i)||/||b|| 2.298968957630e-01
  2 KSP preconditioned resid norm 2.737642092369e+00 true resid norm 4.363156875160e+00 ||r(i)||/||b|| 4.836356076956e-02
  3 KSP preconditioned resid norm 9.030934973388e-01 true resid norm 1.049901550244e+00 ||r(i)||/||b|| 1.163766943984e-02
  4 KSP preconditioned resid norm 4.684585322028e-01 true resid norm 5.441092143035e-01 ||r(i)||/||b|| 6.031197090589e-03
  5 KSP preconditioned resid norm 2.521808331404e-01 true resid norm 1.248737231005e-01 ||r(i)||/||b|| 1.384167030545e-03
  6 KSP preconditioned resid norm 2.271098855110e-01 true resid norm 1.158807467332e-01 ||r(i)||/||b|| 1.284484078159e-0

 90 KSP preconditioned resid norm 9.660040115604e-05 true resid norm 7.174915065593e-05 ||r(i)||/||b|| 7.953059005666e-07
 91 KSP preconditioned resid norm 8.889240149901e-05 true resid norm 6.766515998820e-05 ||r(i)||/||b|| 7.500367671175e-07
 92 KSP preconditioned resid norm 8.754058309734e-05 true resid norm 5.959987075242e-05 ||r(i)||/||b|| 6.606367942906e-07
 93 KSP preconditioned resid norm 8.539649721132e-05 true resid norm 6.814100587768e-05 ||r(i)||/||b|| 7.553112970625e-07
 94 KSP preconditioned resid norm 7.817027748149e-05 true resid norm 6.684155830457e-05 ||r(i)||/||b|| 7.409075262454e-07
 95 KSP preconditioned resid norm 7.636046795911e-05 true resid norm 5.740803316834e-05 ||r(i)||/||b|| 6.363412960476e-07
 96 KSP preconditioned resid norm 7.440847284952e-05 true resid norm 6.343558464168e-05 ||r(i)||/||b|| 7.031538953451e-07
 97 KSP preconditioned resid norm 6.674922536376e-05 true resid norm 5.853965992631e-05 ||r(i)||/||b|| 6.488848513319e-07
 98 KSP preconditioned r